This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-model-cookbooks/demand_curve).

## Demand Curve Pipeline Tutorial

This worksheet demonstrates a Wallaroo pipeline with data preprocessing, a model, and data postprocessing.

The model is a "demand curve" that predicts the expected number of units of a product that will be sold to a customer as a function of unit price and facts about the customer. Such models can be used for price optimization or sales volume forecasting.  This is purely a "toy" demonstration, but is useful for detailing the process of working with models and pipelines.

Data preprocessing is required to create the features used by the model. Simple postprocessing prevents nonsensical estimates (e.g. negative units sold).

## Prerequisites

* An installed Wallaroo instance.
* The following Python libraries installed:
  * `os`
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.

In [42]:
import json
import wallaroo
from wallaroo.object import EntityNotFoundError
import pandas
import numpy
import conversion
from wallaroo.object import EntityNotFoundError
import pyarrow as pa

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

# ignoring warnings for demonstration
import warnings
warnings.filterwarnings('ignore')

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [43]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

Now that the Wallaroo client has been initialized, we can create the workspace and call it `demandcurveworkspace`, then set it as our current workspace.  We'll also create our pipeline so it's ready when we add our models to it.

We'll set some variables and methods to create our workspace, pipelines and models.  Note that as of the July 2022 release of Wallaroo, workspace names must be unique.  Pipelines with the same name will be created as a new version when built.

In [44]:
workspace_name = 'demandcurveworkspace'
pipeline_name = 'demandcurvepipeline'
model_name = 'demandcurvemodel'
model_file_name = './demand_curve_v1.onnx'

In [45]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(name)
    return pipeline

In [46]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

demandcurve_pipeline = get_pipeline(pipeline_name)
demandcurve_pipeline

name,demandcurvepipeline
created,2023-09-01 19:35:46.308437+00:00
last_updated,2023-09-07 15:22:17.833035+00:00
deployed,True
tags,
versions,"e17fe1ea-5d90-4679-8d7a-adf2bae89fdc, f12fc645-3d3d-45bb-a498-8455f615f05a, abc70819-4833-4cec-bf84-20fbf590f2d2, 6c243a4e-31c0-4d95-ba5d-a1dffdc452cc, 980c0707-2276-459e-8bc1-97469373a9d5, 45953caa-e940-4ace-bc1e-d8871c25db13, 0bb8d859-dd41-4de6-b61f-721208ad2d14, bfbea1cf-0f56-46f8-9def-4fa8a4dcb0a6, 78bc4874-1ee4-4d93-b6e5-76baa446f8bb, 2ea4da00-7521-4ffe-9e74-7b41a440021a, 0304ba8d-d1ff-47e6-acfc-b68454d09126, e4607e83-d2e1-4809-9047-a5f5d5d27468, 4d0da913-59fb-4e99-9d6b-1c18377ab34c, 9d5076cb-5752-43ab-af4f-4c904d06ba52, d06a9550-d693-445a-a6cd-b55635018c57, c95ee014-f431-4d8d-ab99-669fde88173d, bd0c6894-63bc-4cb9-b378-390640307a36, 4136fa8b-270e-49d6-836e-75f7c9c73def, 00a4fb64-302d-4a27-90ac-86e08fb6afe8, 94f0129f-f9c1-474a-89fa-7110c81c9a22, 701a759d-6d45-4d61-bfd0-b878d9a09fc3, 4a85b06f-9051-48ff-9f9e-6b6e4e4b8d68, 698c13e2-4120-4132-b72b-99cd8c0fdd99, 1314a1a4-eabc-437d-bf3f-83f54686970e"
steps,preprocess
published,False


With our workspace established, we'll upload two models:

* `demand_curve_v1.onnx`: Our demand_curve model.  We'll store the upload configuration into `demand_curve_model`.
* `postprocess.py`: A Python step that will zero out any negative values and return the output variable as "prediction".

Note that the order we upload our models isn't important - we'll be establishing the actual process of moving data from one model to the next when we set up our pipeline.

In [47]:
# upload to wallaroo
demand_curve_model = wl.upload_model(model_name, model_file_name, framework=wallaroo.framework.Framework.ONNX).configure()

In [7]:
# load the preprocess module
module_pre = wl.upload_model("preprocess", "./preprocess.py", framework=wallaroo.framework.Framework.PYTHON).configure('python')

In [8]:
# load the postprocess module
module_post = wl.upload_model("postprocess", "./postprocess.py", framework=wallaroo.framework.Framework.PYTHON).configure('python')

With our models uploaded, we're going to create our own pipeline and give it three steps:

* First, we apply the data to our `demand_curve_model`.
* And finally, we prepare our data for output with the `module_post`.

In [48]:
# now make a pipeline
demandcurve_pipeline.undeploy()
demandcurve_pipeline.clear()
demandcurve_pipeline.add_model_step(preprocess_step)
demandcurve_pipeline.add_model_step(demand_curve_model)
demandcurve_pipeline.add_model_step(step)

name,demandcurvepipeline
created,2023-09-01 19:35:46.308437+00:00
last_updated,2023-09-07 15:22:17.833035+00:00
deployed,False
tags,
versions,"e17fe1ea-5d90-4679-8d7a-adf2bae89fdc, f12fc645-3d3d-45bb-a498-8455f615f05a, abc70819-4833-4cec-bf84-20fbf590f2d2, 6c243a4e-31c0-4d95-ba5d-a1dffdc452cc, 980c0707-2276-459e-8bc1-97469373a9d5, 45953caa-e940-4ace-bc1e-d8871c25db13, 0bb8d859-dd41-4de6-b61f-721208ad2d14, bfbea1cf-0f56-46f8-9def-4fa8a4dcb0a6, 78bc4874-1ee4-4d93-b6e5-76baa446f8bb, 2ea4da00-7521-4ffe-9e74-7b41a440021a, 0304ba8d-d1ff-47e6-acfc-b68454d09126, e4607e83-d2e1-4809-9047-a5f5d5d27468, 4d0da913-59fb-4e99-9d6b-1c18377ab34c, 9d5076cb-5752-43ab-af4f-4c904d06ba52, d06a9550-d693-445a-a6cd-b55635018c57, c95ee014-f431-4d8d-ab99-669fde88173d, bd0c6894-63bc-4cb9-b378-390640307a36, 4136fa8b-270e-49d6-836e-75f7c9c73def, 00a4fb64-302d-4a27-90ac-86e08fb6afe8, 94f0129f-f9c1-474a-89fa-7110c81c9a22, 701a759d-6d45-4d61-bfd0-b878d9a09fc3, 4a85b06f-9051-48ff-9f9e-6b6e4e4b8d68, 698c13e2-4120-4132-b72b-99cd8c0fdd99, 1314a1a4-eabc-437d-bf3f-83f54686970e"
steps,postprocess
published,False


And with that - let's deploy our model pipeline.  This usually takes about 45 seconds for the deployment to finish.

In [49]:
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("1Gi").build()
demandcurve_pipeline.deploy(deployment_config=deploy_config)

name,demandcurvepipeline
created,2023-09-01 19:35:46.308437+00:00
last_updated,2023-09-07 15:24:30.816374+00:00
deployed,True
tags,
versions,"8390a910-f035-4910-8715-7e41fb0892d9, e17fe1ea-5d90-4679-8d7a-adf2bae89fdc, f12fc645-3d3d-45bb-a498-8455f615f05a, abc70819-4833-4cec-bf84-20fbf590f2d2, 6c243a4e-31c0-4d95-ba5d-a1dffdc452cc, 980c0707-2276-459e-8bc1-97469373a9d5, 45953caa-e940-4ace-bc1e-d8871c25db13, 0bb8d859-dd41-4de6-b61f-721208ad2d14, bfbea1cf-0f56-46f8-9def-4fa8a4dcb0a6, 78bc4874-1ee4-4d93-b6e5-76baa446f8bb, 2ea4da00-7521-4ffe-9e74-7b41a440021a, 0304ba8d-d1ff-47e6-acfc-b68454d09126, e4607e83-d2e1-4809-9047-a5f5d5d27468, 4d0da913-59fb-4e99-9d6b-1c18377ab34c, 9d5076cb-5752-43ab-af4f-4c904d06ba52, d06a9550-d693-445a-a6cd-b55635018c57, c95ee014-f431-4d8d-ab99-669fde88173d, bd0c6894-63bc-4cb9-b378-390640307a36, 4136fa8b-270e-49d6-836e-75f7c9c73def, 00a4fb64-302d-4a27-90ac-86e08fb6afe8, 94f0129f-f9c1-474a-89fa-7110c81c9a22, 701a759d-6d45-4d61-bfd0-b878d9a09fc3, 4a85b06f-9051-48ff-9f9e-6b6e4e4b8d68, 698c13e2-4120-4132-b72b-99cd8c0fdd99, 1314a1a4-eabc-437d-bf3f-83f54686970e"
steps,postprocess
published,False


We can check the status of our pipeline to make sure everything was set up correctly:

In [50]:
demandcurve_pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.3.145',
   'name': 'engine-7d76ccc85d-2q6th',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'demandcurvepipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'curve-preprocess',
      'version': 'a8ab9e19-d2fc-4fcf-8a50-2570f824260c',
      'sha': '3b2b32fb6408ccee5a1886ef5cdb493692080ff6699f49de518b20d9a6f6a133',
      'status': 'Running'},
     {'name': 'curve-postprocess',
      'version': 'd0de3235-f2c7-413d-81e7-ca20e794ca45',
      'sha': 'ecf1a555bb27bcf62bfa42922cf69db23e7188f456015fe8299f02867c3167b2',
      'status': 'Running'},
     {'name': 'demandcurvemodel',
      'version': 'fb9875eb-9707-44d5-82bf-0136ec36dff1',
      'sha': '2820b42c9e778ae259918315f25afc8685ecab9967bad0a3d241e6191b414a0d',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.4.178',
   'name': 'engine-lb-584f54c899-q87zb',
   'status': 'R

Everything is ready.  Let's feed our pipeline some data.  We have some information prepared with the `daily_purchasses.csv` spreadsheet.  We'll start with just one row to make sure that everything is working correctly.

In [51]:
# read in some purchase data
purchases = pandas.read_csv('daily_purchases.csv')

# start with a one-row data frame for testing
subsamp_raw = purchases.iloc[0:1,: ]
subsamp_raw

,Date,cust_known,StockCode,UnitPrice,UnitsSold
0,2010-12-01,False,21928,4.21,1


In [52]:
result = demandcurve_pipeline.infer(subsamp_raw)
display(result)

,time,in.Date,in.StockCode,in.UnitPrice,in.UnitsSold,in.cust_known,out.prediction,check_failures
0,2023-09-07 15:24:54.698,2010-12-01,21928,4.21,1,False,[6.68025518653071],0


We can see from the `out.prediction` field that the demand curve has a predicted slope of 6.68 from our sample data.  We can isolate that by specifying just the data output below.

In [53]:
display(result.loc[0, ['out.prediction']][0])

[6.68025518653071]

# Bulk Inference

The initial test went perfectly.  Now let's throw some more data into our pipeline.  We'll draw 10 random rows from our spreadsheet, perform an inference from that, and then display the results and the logs showing the pipeline's actions.

In [54]:
ix = numpy.random.choice(purchases.shape[0], size=10, replace=False)
converted = conversion.pandas_to_dict(purchases.iloc[ix,: ])
test_df = pd.DataFrame(converted['query'], columns=converted['colnames'])
display(test_df)

output = demandcurve_pipeline.infer(test_df)
display(output)

,Date,cust_known,StockCode,UnitPrice,UnitsSold
0,2010-12-20,False,20712,4.21,1
1,2011-01-27,False,21928,4.13,2
2,2011-09-30,False,23200,4.13,1
3,2011-05-24,False,85099F,4.13,2
4,2011-03-07,False,85099F,4.13,1
5,2011-08-03,False,23203,4.13,2
6,2011-05-20,True,21930,2.08,2
7,2011-04-26,True,21930,2.08,17
8,2011-10-10,False,20712,4.13,13
9,2011-10-13,True,21929,2.08,20


,time,in.Date,in.StockCode,in.UnitPrice,in.UnitsSold,in.cust_known,out.prediction,check_failures
0,2023-09-07 15:24:55.155,2010-12-20,20712,4.21,1,False,[6.68025518653071],0
1,2023-09-07 15:24:55.155,2011-01-27,21928,4.13,2,False,[6.771545926800889],0
2,2023-09-07 15:24:55.155,2011-09-30,23200,4.13,1,False,[6.771545926800889],0
3,2023-09-07 15:24:55.155,2011-05-24,85099F,4.13,2,False,[6.771545926800889],0
4,2023-09-07 15:24:55.155,2011-03-07,85099F,4.13,1,False,[6.771545926800889],0
5,2023-09-07 15:24:55.155,2011-08-03,23203,4.13,2,False,[6.771545926800889],0
6,2023-09-07 15:24:55.155,2011-05-20,21930,2.08,2,True,[33.125323160373426],0
7,2023-09-07 15:24:55.155,2011-04-26,21930,2.08,17,True,[33.125323160373426],0
8,2023-09-07 15:24:55.155,2011-10-10,20712,4.13,13,False,[6.771545926800889],0
9,2023-09-07 15:24:55.155,2011-10-13,21929,2.08,20,True,[33.125323160373426],0


## Undeploy the Pipeline

Once we've finished with our demand curve demo, we'll undeploy the pipeline and give the resources back to our Kubernetes cluster.

In [55]:
demandcurve_pipeline.undeploy()

name,demandcurvepipeline
created,2023-09-01 19:35:46.308437+00:00
last_updated,2023-09-07 15:24:30.816374+00:00
deployed,False
tags,
versions,"8390a910-f035-4910-8715-7e41fb0892d9, e17fe1ea-5d90-4679-8d7a-adf2bae89fdc, f12fc645-3d3d-45bb-a498-8455f615f05a, abc70819-4833-4cec-bf84-20fbf590f2d2, 6c243a4e-31c0-4d95-ba5d-a1dffdc452cc, 980c0707-2276-459e-8bc1-97469373a9d5, 45953caa-e940-4ace-bc1e-d8871c25db13, 0bb8d859-dd41-4de6-b61f-721208ad2d14, bfbea1cf-0f56-46f8-9def-4fa8a4dcb0a6, 78bc4874-1ee4-4d93-b6e5-76baa446f8bb, 2ea4da00-7521-4ffe-9e74-7b41a440021a, 0304ba8d-d1ff-47e6-acfc-b68454d09126, e4607e83-d2e1-4809-9047-a5f5d5d27468, 4d0da913-59fb-4e99-9d6b-1c18377ab34c, 9d5076cb-5752-43ab-af4f-4c904d06ba52, d06a9550-d693-445a-a6cd-b55635018c57, c95ee014-f431-4d8d-ab99-669fde88173d, bd0c6894-63bc-4cb9-b378-390640307a36, 4136fa8b-270e-49d6-836e-75f7c9c73def, 00a4fb64-302d-4a27-90ac-86e08fb6afe8, 94f0129f-f9c1-474a-89fa-7110c81c9a22, 701a759d-6d45-4d61-bfd0-b878d9a09fc3, 4a85b06f-9051-48ff-9f9e-6b6e4e4b8d68, 698c13e2-4120-4132-b72b-99cd8c0fdd99, 1314a1a4-eabc-437d-bf3f-83f54686970e"
steps,postprocess
published,False


Thank you for being a part of this demonstration.  If you have additional questions, please feel free to contact us at Wallaroo.